In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/Spam_SMS.csv')

In [ ]:
df.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5574, 2)

In [ ]:
X=list(df['Message'])

In [ ]:
y=df['Class']

In [ ]:
y.sample(10)

,Class
4600,ham
3632,ham
4550,ham
4083,ham
1134,ham
602,ham
5565,ham
3014,spam
4635,ham
3724,ham


In [ ]:
y=list(y.replace({'spam':0,'ham':1}))

/tmp/ipython-input-1555799946.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y=list(y.replace({'spam':0,'ham':1}))


In [ ]:
y[5]

0

In [ ]:
# if we have multiple class or for even this one we cam also do
#y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
X_train

['Hi:)cts employee how are you?',
 'Sorry pa, i dont knw who ru pa?',
 'ok....take care.umma to you too...',
 'Yeah whatever lol',
 "Nah, I'm a perpetual DD",
 "Babe, I'm back ... Come back to me ...",
 'Nt yet chikku..simple habba..hw abt u?',
 'K k pa Had your lunch aha.',
 'Can ü all decide faster cos my sis going home liao..',
 'Dear good morning now only i am up',
 'Oh k.i think most of wi and nz players unsold.',
 'What Today-sunday..sunday is holiday..so no work..',
 '2marrow only. Wed at  &lt;#&gt;  to 2 aha.',
 'Ok. I asked for money how far',
 'Its ok..come to my home it vl nice to meet and v can chat..',
 'Guy, no flash me now. If you go call me, call me. How madam. Take care oh.',
 'Dear Voucher holder Have your next meal on us. Use the following link on your pc 2 enjoy a 2 4 1 dining experiencehttp://www.vouch4me.com/etlp/dining.asp',
 '7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st "Ur Lovely Friendship"... good

In [ ]:
!pip install transformers

In [ ]:
#tokenizer call

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_encoding=tokenizer(X_train,truncation=True,padding=True,return_tensors="tf")
test_encoding=tokenizer(X_test,truncation=True,padding=True,return_tensors="tf")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
train_encoding[0]

Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
import tensorflow as tf

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    y_train
)).batch(8)
test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
    y_test
)).batch(8)

In [ ]:
model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train model
model.fit(train_dataset, epochs=2, validation_data=test_dataset)

# Evaluate
model.evaluate(test_dataset)

Epoch 1/2
488/488 [==============================] - 150s 253ms/step - loss: 0.0688 - accuracy: 0.9810 - val_loss: 0.0688 - val_accuracy: 0.9863
Epoch 2/2
210/210 [==============================] - 15s 73ms/step - loss: 0.0329 - accuracy: 0.9910


[0.032910823822021484, 0.9910340905189514]

In [ ]:
predictions=model.predict(test_dataset)

210/210 [==============================] - 14s 66ms/step


In [ ]:
logits = predictions.logits
predicted_classes = tf.argmax(logits, axis=1)

In [ ]:
print(predicted_classes)

tf.Tensor([1 1 1 ... 0 1 1], shape=(1673,), dtype=int64)


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,predicted_classes)
cm

array([[ 231,   11],
       [   4, 1427]])

In [ ]:
print(accuracy_score(y_test,predicted_classes))

0.9910340705319785


old version in pytorch of transformer implementation

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args=TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weigth_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
with training_args.strategy.scope():
  model=TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer=TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()
trainer.evaluate(test_dataset)
output=trainer.predict(test_dataset)[1]
